# Read into geopandas dataframe

In [1]:
import os
from pyrosm import OSM

In [2]:
data_dir = "/home/ross/repos/fell_finder/data"

In [3]:
data = OSM(os.path.join(data_dir, 'extracts/osm/hampshire-latest.osm.pbf'))

In [4]:
help(data)

Help on OSM in module pyrosm.pyrosm object:

class OSM(builtins.object)
 |  OSM(filepath, bounding_box=None)
 |
 |  OpenStreetMap PBF reader object.
 |
 |  Parameters
 |  ----------
 |
 |  filepath : str
 |      Filepath to input OSM dataset ( *.osm.pbf )
 |
 |  bounding_box : list | shapely geometry
 |      Filtering OSM data spatially is allowed by passing a
 |      bounding box either as a list `[minx, miny, maxx, maxy]` or
 |      as a Shapely Polygon/MultiPolygon or closed LineString/LinearRing.
 |
 |  Methods defined here:
 |
 |  __getattribute__(self, name)
 |      Return getattr(self, name).
 |
 |  __init__(self, filepath, bounding_box=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  get_boundaries(self, boundary_type='administrative', name=None, custom_filter=None, extra_attributes=None, timestamp=None)
 |      Parses boundaries from OSM.
 |
 |      Parameters
 |      ----------
 |
 |      boundary_type : str
 |          The type of boundarie

# Extract street information

In [5]:
help(data.get_network)

Help on method get_network in module pyrosm.pyrosm:

get_network(network_type='walking', extra_attributes=None, nodes=False, timestamp=None) method of pyrosm.pyrosm.OSM instance
    Parses street networks from OSM
    for walking, driving, and cycling.

    Parameters
    ----------

    network_type : str
        What kind of network to parse.
        Possible values are:
          - `'walking'`
          - `'cycling'`
          - `'driving'`
          - `'driving+service'`
          - `'all'`.

    extra_attributes : list (optional)
        Additional OSM tag keys that will be converted into columns in the resulting GeoDataFrame.

    nodes : bool (default: False)
        If True, 1) the nodes associated with the network will be returned in addition to edges,
        and 2) every segment of a road constituting a way is parsed as a separate row
        (to enable full connectivity in the graph).

    timestamp: str | datetime | int
        If provided, the data from given moment of ti

In [6]:
# Runs in c. 40s
nodes, ways = data.get_network('walking', nodes=True)

In [8]:
ways['oneway'].unique()

array([None, 'yes', 'no', '-1', 'alternating'], dtype=object)

In [ ]:
nodes.head()

,lat,version,timestamp,lon,changeset,visible,tags,id,geometry
0,50.997253,2,1337542012,-1.289405,0,False,None,630299,POINT (-1.2894 50.99725)
1,50.996162,1,1355079684,-1.290058,0,False,None,2058592714,POINT (-1.29006 50.99616)
2,50.995075,1,1142464874,-1.290566,0,False,None,630302,POINT (-1.29057 50.99508)
3,50.994495,1,1355079684,-1.290584,0,False,None,2058592713,POINT (-1.29058 50.9945)
4,50.994144,1,1355079684,-1.290665,0,False,None,2058592712,POINT (-1.29066 50.99414)


In [ ]:
ways.head()

,access,area,bicycle,bridge,busway,cycleway,est_width,foot,footway,highway,int_ref,junction,lanes,lit,maxspeed,motorcar,motorroad,motor_vehicle,name,oneway,overtaking,path,passing_places,psv,ref,service,segregated,sidewalk,smoothness,surface,tracktype,tunnel,width,id,timestamp,version,tags,osm_type,geometry,u,v,length
0,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.2894 50.99725, -1.29006 50.99616)",630299,2058592714,129.631
1,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29006 50.99616, -1.29057 50.99508)",2058592714,630302,126.025
2,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29057 50.99508, -1.29058 50.9945)",630302,2058592713,64.487
3,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29058 50.9945, -1.29066 50.99414)",2058592713,2058592712,39.431
4,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29066 50.99414, -1.29065 50.99385)",2058592712,630303,32.245


In [ ]:
ways.loc[ways['id']==130, :].head()

,access,area,bicycle,bridge,busway,cycleway,est_width,foot,footway,highway,int_ref,junction,lanes,lit,maxspeed,motorcar,motorroad,motor_vehicle,name,oneway,overtaking,path,passing_places,psv,ref,service,segregated,sidewalk,smoothness,surface,tracktype,tunnel,width,id,timestamp,version,tags,osm_type,geometry,u,v,length
0,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.2894 50.99725, -1.29006 50.99616)",630299,2058592714,129.631
1,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29006 50.99616, -1.29057 50.99508)",2058592714,630302,126.025
2,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29057 50.99508, -1.29058 50.9945)",630302,2058592713,64.487
3,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29058 50.9945, -1.29066 50.99414)",2058592713,2058592712,39.431
4,None,None,designated,None,None,None,None,designated,None,bridleway,None,None,None,no,None,None,None,None,Castle Lane,None,None,None,None,None,None,None,None,None,None,unpaved,None,None,None,130,1614378250,16,"{""visible"":false,""designation"":""public_bridlew...",way,"LINESTRING (-1.29066 50.99414, -1.29065 50.99385)",2058592712,630303,32.245


# Generate Nodes/Edges tables

In [ ]:
nodes = nodes.loc[:, ['id', 'lat', 'lon']]

In [ ]:
ways = ways.loc[:, ['id', 'u', 'v', 'length', 'highway', 'surface', 'bridge']]

In [39]:
ways.loc[:, 'row_no'] = ways.index

In [40]:
ways.head()

,id,u,v,length,highway,surface,bridge,row_no
0,130,630299,2058592714,129.631,bridleway,unpaved,None,0
1,130,2058592714,630302,126.025,bridleway,unpaved,None,1
2,130,630302,2058592713,64.487,bridleway,unpaved,None,2
3,130,2058592713,2058592712,39.431,bridleway,unpaved,None,3
4,130,2058592712,630303,32.245,bridleway,unpaved,None,4


# Convert to Polars

In [43]:
import polars as pl

In [44]:
nodes = pl.DataFrame(nodes)

In [45]:
ways = pl.DataFrame(ways)

# Assign bng coords & partitions

In [46]:
from bng_latlon import WGS84toOSGB36

In [47]:
WGS84toOSGB36(0.0, 0.0)

(622674.8553, -5527598.33)

In [49]:
nodes = nodes.select(
    'id',
    'lat',
    'lon',
    pl.struct(
        'lat', 'lon'
    ).map_elements(
        lambda x: WGS84toOSGB36(x['lat'], x['lon']),
        return_dtype=pl.List(pl.Float64)
    ).alias('bng_coords')
)

In [54]:
nodes.select('id', 'lat', 'lon', pl.col('bng_coords').list.get(0).alias('easting'), pl.col('bng_coords').list.get(1).alias('northing'))

id,lat,lon,easting,northing
i64,f64,f64,f64,f64
630299,50.997253,-1.289405,449962.7475,122224.5742
2058592714,50.996162,-1.290058,449918.1061,122102.8077
630302,50.995075,-1.290566,449883.5643,121981.5632
2058592713,50.994495,-1.290584,449882.9296,121917.0705
2058592712,50.994144,-1.290665,449877.6412,121877.9884
…,…,…,…,…
6274955096,51.348545,-0.883894,477826.2559,161642.7053
658993503,51.348621,-0.884071,477813.8234,161651.0027
3976775800,51.348686,-0.884224,477803.0084,161658.0518


# Tag edges with lat/lon coords

In [56]:
ways = ways.select(
    pl.col('u').alias('src'),
    pl.col('v').alias('dst'),
    pl.col('id').alias('way_id'),
    'length',
    'highway',
    'surface',
    'bridge',
    'row_no'
)

* Convert row_no to way_inx (groupby way_id, orderby row_no)
* Join on start lat/lon
* Assign easting/northing as with nodes

In [57]:
ways.count()

src,dst,way_id,length,highway,surface,bridge,row_no
u32,u32,u32,u32,u32,u32,u32,u32
1345330,1345330,1345330,1345330,1345330,528830,6098,1345330
